In [1]:
from dotenv import load_dotenv
import os
import subprocess
import json
from constants import *
from web3 import Web3
from eth_account import Account
from web3.middleware import geth_poa_middleware
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI

In [2]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

load_dotenv()
mnemonic = os.getenv("mnemonic")

In [3]:
def derive_wallets(coin):
    command = f'php derive -g --mnemonic="{mnemonic}" --cols=address,index,path,privkey,pubkey --coin="{coin}" --numderive=3 --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    return keys

coins = {
    ETH: derive_wallets(ETH),
    BTCTEST: derive_wallets(BTCTEST)
}
print(coins)

{'eth': [{'address': '0x2C145221e72aEe2db8a13d0972924e40f9AEecaA', 'index': 0, 'path': "m/44'/60'/0'/0/0", 'privkey': '0xab5bef9b31b0ad43ed6a78e1ab7a40d7d56fc7b1c3aacf17e052058744f50df4', 'pubkey': '03052ded438c70d678c11066c1071c8655f4e0fb9e10d36d6cabc9b25cb31a1d55'}, {'address': '0xedE4f14976fb94D5D189EB34E9DA203460797Bb4', 'index': 1, 'path': "m/44'/60'/0'/0/1", 'privkey': '0x88acfd865c254efbef1b9c56450830a3122d420aa618de3f8a41757f0936db38', 'pubkey': '0289aae07a61b9a909c51f17e977a1928537a8bbdba965eeb4b3fa9fa86adb6c2e'}, {'address': '0xB5f6e75B3F4665c34988A35f7bb7D8704dA6a728', 'index': 2, 'path': "m/44'/60'/0'/0/2", 'privkey': '0xdd4dbfb8721f57f4158ee0362d201f893ec1f0362c4a40bc0f7bacd5a4c922cd', 'pubkey': '03b2db9c0ad33cd5aebb9edd04cb8578433598cd78e090b96e7842894cdfd87555'}], 'btc-test': [{'address': 'n2i28W1PioB1Y98YdR9nghLaYNcZT13XSJ', 'index': 0, 'path': "m/44'/1'/0'/0/0", 'privkey': 'cTwj6dxQaBd9QmTxCMdFAANDMzwi6NWsvhdZHUEnLN9opbhhS9Qi', 'pubkey': '03b9c5655d5740758a09321be11704

In [4]:
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [5]:
print(w3.eth.blockNumber)

120


In [6]:
def create_tx(coin, account, to, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas({"from": account.address, "to": to, "value": amount})
        return {
            "from": account.address,
            "to": to,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [7]:
def send_tx(coin, account, to, amount):
    tx = create_tx(coin, account, to, amount)
    signed_tx = account.sign_transaction(tx)
    if coin == ETH:
        return w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    elif coin == BTCTEST:
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

In [8]:
eth_account_one = priv_key_to_account(ETH, coins[ETH][0]['privkey'])
eth_account_one.address

'0x2C145221e72aEe2db8a13d0972924e40f9AEecaA'

In [9]:
btc_account_one = priv_key_to_account(BTCTEST, coins[BTCTEST][0]['privkey'])
btc_account_one.address

'n2i28W1PioB1Y98YdR9nghLaYNcZT13XSJ'

In [10]:
btc_address_two = print(coins[BTCTEST][1]['address'])

mgiZnrBsTeFvnjn2w4mDzPGEqmdFhoXt2N


In [11]:
eth_address_two = print(coins[ETH][1]['address'])

0xedE4f14976fb94D5D189EB34E9DA203460797Bb4


In [12]:
print(w3.eth.getBalance(eth_account_one.address))

904625697166532776746648320380374280103671755200315806558161352061821325312


In [13]:
#send_tx(BTCTEST, btc_account_one, btc_address_two, 0.0002)

In [14]:
send_tx(ETH, eth_account_one, eth_address_two, 1000000000000)

HexBytes('0x31f6094dbd755629ffdfd2e2489896d69c4795d8ab6ea15e7c946f6929fc3677')